In [2]:
import geopandas
import geoplot
import pandas as pd
from shapely.geometry import LineString, Point, Polygon
from shapely.geometry import LineString, Point

world = geopandas.read_file(
    geopandas.datasets.get_path('naturalearth_lowres')
)

In [3]:
a = geopandas.read_file('D:\DS\COVID-19\gadm36_BHR_shp\gadm36_BHR_0.shp') #BHR
b = geopandas.read_file('D:\DS\COVID-19\gadm36_GUF_shp\gadm36_GUF_0.shp') #French Guinea
c = geopandas.read_file('D:\DS\COVID-19\gadm36_GLP_shp\gadm36_GLP_0.shp') #Guadeloupe
d = geopandas.read_file('D:\DS\COVID-19\gadm36_SGP_shp\gadm36_SGP_0.shp') #Singapore
e = geopandas.read_file('D:\DS\COVID-19\gadm36_MDV_shp\gadm36_MDV_0.shp') #Maldives
f = geopandas.read_file('D:\DS\COVID-19\gadm36_MLT_shp\gadm36_MLT_0.shp') #Malta
g = geopandas.read_file('D:\DS\COVID-19\gadm36_MCO_shp\gadm36_MCO_0.shp') #Monaco
h = geopandas.read_file('D:\DS\COVID-19\gadm36_LIE_shp\gadm36_LIE_0.shp') #Liechtenstein
i = geopandas.read_file('D:\DS\COVID-19\gadm36_SMR_shp\gadm36_SMR_0.shp') #Liechtenstein
j = geopandas.read_file('D:\DS\COVID-19\gadm36_REU_shp\gadm36_REU_0.shp') #Reunion

In [14]:
world = world.append(pd.Series([1493000, "Asia", 'Bahrain', 'BHR', 21810, a['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([3700, "Asia", 'Cruise Ship', 'CRUS', 99, Polygon([(35.4437, 139.638), (35.4438, 139.637), (35.4439, 139.636)])], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([290691, "South America", 'French Guiana', 'FRGF', 18300, b['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([395700, "North America", 'Guadeloupe', 'FRGP', 25479, c['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([5638700, "Asia", 'Singapore', 'SG', 65627, d['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([392473, "Asia", 'Maldives', 'MV', 15463, e['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([493559, "Europe", 'Malta', 'SG', 32021, f['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([38300, "Europe", 'Monaco', 'MC', 185741, g['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([38557, "Europe", 'Liechtenstein', 'LC', 98432, h['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([33344, "Europe", 'San Marino', 'SM', 44947, i['geometry'][0]], index=world.columns ), ignore_index=True)

world = world.append(pd.Series([859959, "Africa", 'Reunion', 'RE', 25900, j['geometry'][0]], index=world.columns ), ignore_index=True)

In [16]:
covdata = pd.read_csv('your file path') #Data

In [20]:
world['pop_est'] = world['pop_est'].astype(int)

In [22]:
covdata['before'] = 0
covdata['confirmed_today'] = 0
tmp_count_prev = 0
count = 0
for x in covdata['Date'].unique():
    count = count + 1
    tmp = covdata[covdata['Date']==x]
    tmp_count = tmp.groupby(['Country/Region'])['Confirmed'].sum()
    tmp_count_copy = tmp_count.copy()
    tmp_count = tmp_count - tmp_count_prev
    tmp_count_prev = tmp_count_copy
    for y in tmp_count.index:
        covdata.loc[(covdata['Date'] == x) & (covdata['Country/Region'] == y), 'confirmed_today'] = tmp_count[y]

In [24]:
tmp_count = covdata.groupby(['Country/Region'])['confirmed_today'].max()

for y in tmp_count_prev.index:
    world.loc[(world['name'] == y), 'world_count'] = tmp_count_prev[y]
    
world['world_count'] = world['world_count'].fillna(0)

In [25]:
import mapclassify
scheme = mapclassify.Percentiles(world['world_count'], pct=[1, 20, 50, 80, 95, 98, 99,  99.5, 100])# scheme.bins
scheme

Percentiles                 

      Interval         Count
----------------------------
[    0.00,     0.00] |    69
(    0.00,     0.00] |     0
(    0.00,     5.00] |    30
(    5.00,    81.20] |    51
(   81.20,   932.30] |    28
(  932.30,  5946.22] |     6
( 5946.22, 12182.48] |     2
(12182.48, 21773.52] |     1
(21773.52, 80945.00] |     1

In [26]:
import numpy as np
sort_lat_long = [x for x in zip(covdata.Long, covdata.Lat)]
sort_lat_long.sort()
def closest(x,y):
    inp_dist = [(x,y) for z in range(len(sort_lat_long))]
    diff = [sum((a-b)**2) for a, b in zip(np.array(inp_dist), np.array(sort_lat_long))]
    return sort_lat_long[diff.index(min(diff))]

In [1]:
import geoplot.crs as gcrs
import matplotlib.animation as animation
import matplotlib.pyplot as plt
fig = plt.figure()
count = 0 
ims = []
ims1 = []
prev_geo = 0
for x in covdata['Date'].unique():
    count = count + 1
    if (count > 100):
        break
    if (count <= 100):
        tmp = (covdata[covdata['Date'] == (x)]).copy()
        tot = tmp['confirmed_today'].sum()
        world['name_count'] = world['name'].apply(lambda y: tmp[tmp['Country/Region']==y]['Confirmed'].sum())
        world['name_count'] = world['name_count'].fillna(0)
        tmp['name_count'] = tmp['Country/Region'].apply(lambda z: tmp[tmp['Country/Region']==z]['confirmed_today'].sum())
        tmp['name_count'] = tmp['name_count'].fillna(0)
        tmp['rate'] = tmp['name_count'] / tot
        tmp['rate'] = tmp['rate'].fillna(0)
        ai = geoplot.choropleth(
            world, hue=world['name_count'] , scheme=scheme,
            cmap='Reds', figsize=(8, 4), legend=True, legend_kwargs={'loc': 'lower left', 'fontsize' : 'xx-small'}, legend_labels=[ 
                '$\mathregular{1^{st}}$ Percentile', '$\mathregular{20^{th}}$  Percentile', '$\mathregular{50^{th}}$ Percentile', 
                '$\mathregular{80^{th}}$ Percentile', '$\mathregular{95^{th}}$ Percentile', '$\mathregular{98^{th}}$ Percentile', 
                '$\mathregular{99^{th}}$ Percentile','$\mathregular{99.5^{th}}$ Percentile', '$\mathregular{100^{th}}$ Percentile']
        )
        
        ai.set_title('Date: '+ x, pad = 1.5, fontsize = 8, color='blue')
        tmp1 = tmp[tmp['Confirmed']!=0].copy()
        atmp = pd.concat([tmp1.groupby(['Country/Region'])['Lat', 'Long'].mean(), tmp1.groupby(['Country/Region'])['rate'].mean()], axis = 1)
        atmp['centroid_lat'] = atmp['Lat']*atmp['rate']
        atmp['centroid_long'] = atmp['Long']*atmp['rate']
        ap = Point(closest(atmp['centroid_long'].sum(),atmp['centroid_lat'].sum()))
        
        if(isinstance(prev_geo, int)):
            df1 = pd.DataFrame({'geometry': [ap]})
            df1['geom_1'] = df1['geometry']
        if(not isinstance(prev_geo, int)):
            df1 = df1.append(pd.Series([ap, prev_geo], index=df1.columns), ignore_index=True)
            
        gdf1 = geopandas.GeoDataFrame(df1.copy())
        
        gdf1.plot(ax=ai, color='black', markersize=10,zorder=2)
       
        prev_geo = ap
        
        lin = [(xy) for xy in zip((gdf1.geometry), (gdf1.geom_1))]
        lins = [LineString(x) for x in lin]
        
        gdf = geopandas.GeoDataFrame(gdf1, geometry=lins)
        gdf.plot(ax=ai, color='blue',linewidths=1.25,linestyles='dashed')